# Analysing ChIP-seq data with BioNumPy

In this task, we will use BioNumPy to analyse the transcription factor CTCF. We will work with a publically available dataset from the Encode Project.

The task is divided into four parts:

1) Filter/analyse the raw FASTQ reads
2) Look for motif matches in the FASTQ reads
3) Analyse the peaks
4) Analyse multiple peak data sets together (multiomics)

## Part 1

Start by downloading raw FASTQ data from ENCODE if you havn't already:


In [ ]:
!wget https://www.encodeproject.org/files/ENCFF001HTO/@@download/ENCFF001HTO.fastq.gz

Use BioNumPy to analyse the raw reads. For instance, try to find the following (or other things you are interested in):

* The total number of reads
* The total number of bases in the reads
* The average read length
* The total number of G and C
* The GC-content
* The average base quality

Feel free to look at code from the previous part if you've forgotten how to do things.

## Part 2: Analysing motif matches

Assume we have used the FASTQ-reads from the previous part to perform peak calling of the CTCF trancsription factor.

We have obtained a narrowPeak file with the called peaks, which are intervals on a reference genome. We also need the hg38 reference genome sequence.

In [ ]:
!wget ..
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

**TASK** Read the file using BioNumPy. Find out which fields your dataset has.